tool_list =  
(
    [add, multiply, search_tool, retriever_tool]  
     + gmail_tool_list + loaded_tool_list
)

* from langgraph.checkpoint.memory import MemorySaver
* DuckDuckGoSearchRun
* from langchain.agents import load_tools
* create_retriever_tool
* ToolNode

### tools 선언

In [11]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """숫자 a와 b를 더합니다."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """숫자 a와 b를 곱합니다."""
    return a * b

In [5]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

In [12]:
from langchain_google_community import GmailToolkit

from langchain_google_community.gmail.utils import (
    build_resource_service,
    get_gmail_credentials,
)

# Can review scopes here https://developers.google.com/gmail/api/auth/scopes
# For instance, readonly scope is 'https://www.googleapis.com/auth/gmail.readonly'
credentials = get_gmail_credentials(
    token_file="google/token.json",
    scopes=["https://mail.google.com/"],
    client_secrets_file="google/credentials.json",
)
api_resource = build_resource_service(credentials=credentials)
gmail_toolkit = GmailToolkit(api_resource=api_resource)
gmail_tool_list = gmail_toolkit.get_tools()

In [8]:
from langchain.agents import load_tools

loaded_tool_list = load_tools(
    ["arxiv"],
)

In [14]:
import os

from langchain_pinecone import PineconeVectorStore
from langchain_upstage import UpstageEmbeddings
from langchain_core.tools.retriever import create_retriever_tool

embedding_function = UpstageEmbeddings(
    model="embedding-query"
)
vector_store = PineconeVectorStore(
    embedding=embedding_function,
    index_name="real-estate",
)
retriever = vector_store.as_retriever(search_kwargs={"k": 3})
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="real_estate_tax_retriever",
    description="Contains information about real estate tax up to December 2024",
)

In [15]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [17]:
from langgraph.prebuilt import ToolNode

tool_list = [retriever_tool, search_tool, add, multiply] + loaded_tool_list + gmail_tool_list

llm_with_tools = llm.bind_tools(tool_list)
tool_node = ToolNode(tool_list)

In [2]:
from langgraph.graph import StateGraph, MessagesState

class AgentState(MessagesState):
    summary: str

graph_builder = StateGraph(AgentState)

In [28]:
from langchain_core.messages import SystemMessage

def agent(state: AgentState):

    messages = state["messages"]
    summary = state["summary"]

    if summary != "":
        messages = [SystemMessage(content=f"""
        이것은 이전 대화들을 요약한 것입니다. 참고하여주세요.\n요약: {summary}
        """)] + messages

    

    response = llm_with_tools.invoke(messages)

    return {"messages": [response]}

In [29]:
query = "최신 ai 논문을 찾아주세요"

In [30]:
initial_state = {"messages": query, "summary": ""}

In [31]:
result = agent(initial_state)

In [23]:
result

{'messages': AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zYAPTvKgUwbjpzFpO4ZXwyL7', 'function': {'arguments': '{"query":"artificial intelligence"}', 'name': 'arxiv'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 744, 'total_tokens': 760, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_46bff0e0c8', 'id': 'chatcmpl-CAzCrJZLqFR97UVlXWVZx6PhkqzxI', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8025cf87-aba1-44db-822e-fcfdd998b9e4-0', tool_calls=[{'name': 'arxiv', 'args': {'query': 'artificial intelligence'}, 'id': 'call_zYAPTvKgUwbjpzFpO4ZXwyL7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 744, 'output_tokens': 16, 'total_tokens'

In [36]:
result['messages'][-1].tool_calls[0]

{'name': 'arxiv',
 'args': {'query': 'latest AI papers'},
 'id': 'call_PeBW9BXMSj3li1VnFTuMsPnY',
 'type': 'tool_call'}

In [38]:
tool_result = tool_node.invoke(result)

In [44]:
tool_result['messages'][0].content

'Published: 2025-03-04\nTitle: Reflection on Data Storytelling Tools in the Generative AI Era from the Human-AI Collaboration Perspective\nAuthors: Haotian Li, Yun Wang, Huamin Qu\nSummary: Human-AI collaborative tools attract attentions from the data storytelling\ncommunity to lower the barrier of expertise and streamline the workflow. The\nrecent advance in large-scale generative AI techniques, e.g., large language\nmodels (LLMs) and text-to-image models, has the potential to enhance data\nstorytelling with their power in visual and narration generation. After two\nyears since these techniques were publicly available, it is important to\nreflect our progress of applying them and have an outlook for future\nopportunities. To achieve the goal, we compare the collaboration patterns of\nthe latest tools with those of earlier ones using a dedicated framework for\nunderstanding human-AI collaboration in data storytelling. Through comparison,\nwe identify persistent collaboration patterns, 